<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Recurrent%20Neural%20Network/Joon_Del.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten, TimeDistributed
from matplotlib import pyplot as plt

In [2]:
batch_size = 12
epoch = 10
lr = 0.001
width = height = 50

In [3]:
path = "/content/drive/MyDrive/joon_del/"
dirs = os.listdir(path)
dirs

['0', '1']

In [4]:
path = "/content/drive/MyDrive/joon_del/"
dirs = os.listdir(path)

File=[]
label=[]

for dir in dirs:
  subfolder = f"/content/drive/MyDrive/joon_del/{dir}"
  files = os.listdir(subfolder)
  for f in files:
    cap = cv2.VideoCapture(f'/content/drive/MyDrive/joon_del/{dir}/{f}')
    video=[]
    while(True):
      ret, frame = cap.read()
      if ret == False:
          break
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      img = cv2.resize(frame_gray, dsize=(width, height))
      video.append(img)
    File.append(video)
    label.append(float(dir))




In [5]:
longest_list = max(len(f) for f in File)
longest_list

313

In [6]:
for i, f in enumerate(File):
  if len (f) < longest_list:
    for j in range(longest_list-len(f)):
      File[i].append(np.zeros((width, height)))

In [7]:
File = np.asarray(File)
label = np.asarray(label)
File = File[..., np.newaxis]
label = label[..., np.newaxis]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(File, label, test_size=0.2, random_state=42)

In [9]:
print('train data size:', X_train.shape)
print('train label size:', y_train.shape)


train data size: (50, 313, 50, 50, 1)
train label size: (50, 1)


In [10]:
crnn = tf.keras.models.Sequential([
                  # CNN
                  TimeDistributed(Conv2D(16, (3, 3), activation="relu", input_shape=(None, width, height, 1))),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
                  TimeDistributed(Conv2D(8, (3, 3), activation="relu")),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
                  TimeDistributed(Conv2D(4, (3, 3), activation="relu")),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
                  TimeDistributed(Conv2D(2, (3, 3), activation="relu")),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                  TimeDistributed(Flatten()),

                  # RNN
                  SimpleRNN(50),
                  Dense(2, activation="softmax")
])

In [11]:
crnn.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

crnn.fit(X_train, y_train, batch_size=batch_size, epochs=epoch)

Epoch 1/10
5/5 [==============================] - 16s 427ms/step - loss: 0.6775 - accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 2s 363ms/step - loss: 0.7626 - accuracy: 0.4400
Epoch 3/10
5/5 [==============================] - 2s 378ms/step - loss: 0.7609 - accuracy: 0.4400
Epoch 4/10
5/5 [==============================] - 2s 394ms/step - loss: 0.6854 - accuracy: 0.5600
Epoch 5/10
5/5 [==============================] - 2s 366ms/step - loss: 0.6975 - accuracy: 0.5200
Epoch 6/10
5/5 [==============================] - 2s 339ms/step - loss: 0.6957 - accuracy: 0.5400
Epoch 7/10
5/5 [==============================] - 2s 351ms/step - loss: 0.6902 - accuracy: 0.5000
Epoch 8/10
5/5 [==============================] - 2s 386ms/step - loss: 0.6902 - accuracy: 0.5600
Epoch 9/10
5/5 [==============================] - 2s 365ms/step - loss: 0.6909 - accuracy: 0.5400
Epoch 10/10
5/5 [==============================] - 2s 360ms/step - loss: 0.6958 - accuracy: 0.5400


In [12]:
crnn_eval = crnn.evaluate(X_test, y_test)


1/1 [==============================] - 1s 782ms/step - loss: 0.7065 - accuracy: 0.4615
